<a href="https://colab.research.google.com/github/fabian819/DSF/blob/main/CodingTask1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Data Science Fundamentals for DCBP, S. Haug, University of Bern 

# Coding Task 1

- Deadline (upload to Ilias): 2022-03-22 17:00
- Estimated effort: 2-5 hours (if you don't progress after 2 hours of trying, search for help, ChatGPT, Google, colleagues, course tutor)
- Data on Ilias: CCD-Data with metadata file CCD-Metadata


## Tasks (maximum 12.5 points)

- **CT-1.1** Write a method (function) which removes the header information in the 
datafile and saves that information into a separate textfile. [0.5 points] 
- **CT-1.2** Write a function which reduces the data resolution by merging/averaging columns such that there is only one column per 1 nm. [2.0 points]  
- **CT-1.3** Now generalize the above merging so that it works for any number nm er column. [2.0 points]
- **CT-1.4** Implement the saving of the reduced dataset to a file. Do this (1) by writing an explicit loop (write line by line) and (2) by using pandas methods. Measure the running times of the two approaches. [2.0 points]
- **CT-1.5** Write a function which takes two wavelengths as input and plots the difference of the data at these lengths over time. [2.0 points] 
- **CT-1.6** Let the x-axis be in seconds, label the axes with names and units, label the plot with color and legend, make a title for the plot. Save the plot to a file. [2.0 points]
- **CT-1.7** Plot the full and some reduced datasets in the same figure. Is there any visual difference? [2.0 points]

All the relevant imports

In [34]:
import pandas as pd
from google.colab import drive
import re
import os
import numpy as np

Mount the google drive

In [23]:
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


**CT-1.1**

In [25]:

def isolate_header(file_path,file_name):
  '''Takes the a file and splits on the line
  which starts with >, only works if > is not
  used in other parts of the file'''

  path = file_path + '/' + file_name

  if  not os.path.exists(path):
    raise Exception('There is no such file')

  with open(path, 'r') as instream:
    separated = re.split('>.*\n', instream.read())

  with open(path, 'w') as outstream:
    outstream.write(separated[-1])

  try:
    with open(file_path + '/CCD-metadata.txt', 'w') as outstream:
      outstream.write(separated[-2])
  except:
    print("Probably already split")

  

In [24]:
isolate_header('/content/drive/MyDrive/DSF','CCD-Data.txt')

**CT-1.2**

In [44]:
df = pd.read_csv('/content/drive/MyDrive/DSF/CCD-Data.txt', sep='\t')
df.head()
renamed = df.rename(columns=lambda s: s.split('.')[0])
renamed.groupby(level=0, axis=1)

In [ ]:
def merging_columns(dataframe):
  